In [ ]:
%%bash
# preamble script to check and install AMUSE components if necessary

# required packages for this tutorial:
PACKAGES="amuse-framework"
# skip in case a full development install is present
pip show amuse-devel && exit 0
for package in ${PACKAGES} 
do
  pip show ${package} || pip install ${package}
done

In [ ]:
# the following fixes are highly recommended

#allow oversubscription for openMPI
import os
os.environ["OMPI_MCA_rmaps_base_oversubscribe"]="true"

# use lower cpu resources for idle codes
from amuse.support import options
options.GlobalOptions.instance().override_value_for_option("polling_interval_in_milliseconds", 10)


In [ ]:
%matplotlib inline
from matplotlib import pyplot
import numpy

In many community codes and for a lot of algorithms it makes sense to use units for which the base units are not (fully) specified. This is often the case when e.g. the equations solved are scale free or for initial conditions where similar models can be scaled to different sizes. Although the quantities involved do not have a specific unit base, they still have a dimension (mass, length etc.). In AMUSE we can use *generic units* in this case. In other words, you can specify if a value has a *mass*, *length* or *time* dimension, or any combination thereof, such as *length* per *time*.

First import everything from amuse.lab:

In [ ]:
from amuse.lab import *

AMUSE includes two generic unit systems, the **generic_unit_system** is the most general, the **nbody_system** is a special case and always defines the gravitational constant to be `G=1`. For gravity calculations the **nbody_system** module is recommended as this follows the general practice in most n-body codes.

The generic units are defined in the **generic_system** and **nbody_system** modules.

In [ ]:
print(10.0 | nbody_system.length)
print(10.0 | generic_unit_system.length)

Quantities with generic units work exactly the same as quantities with normal (**S.I.**) units.

In [ ]:
cluster_mass = 1.0 | generic_unit_system.length
mean_speed = 0.1 | generic_unit_system.length / generic_unit_system.time
print(mean_speed * cluster_mass)


Generic quantities are very useful and can be applied almost everywhere in AMUSE. 

To convert to a specific system of units you'll need a converter. For nbody units you can create a converter like this:

In [ ]:
converter = nbody_system.nbody_to_si(1 | units.MSun, 1 | units.AU)

An ``nbody_system`` converter always needs two orthogonal quantities apart from `G=1` which is already defined. These quantities can be simple (like 1 solar mass) or combined (like  10 km/s). The two quantities fix the scaling and will be used to convert to and from the nbody units:

In [ ]:
print("Mass of the sun, scaled:", converter.to_nbody(1 | units.MSun))
print("10 nbody masses, in S.I.:", converter.to_si(10 | nbody_system.mass))
print("1 nbody time, in S.I:", converter.to_si(1 | nbody_system.time).in_(units.yr))
print("10 km/s, in nbody:", converter.to_nbody(10.0 | units.km / units.s))

For the generic unit converter, you can specify up to 7 quantities (as there are 7 base properties). Any combination of quantities is possible as long as it results in a orthogonal set of converters. 

In [ ]:
converter = generic_unit_converter.ConvertBetweenGenericAndSiUnits(1 | units.MSun, 1 | units.AU, constants.G)
print("Mass of the sun, scaled:", converter.to_nbody(1 | units.MSun))
print("10 generic masses, in S.I.:", converter.to_si(10 | nbody_system.mass))
print("1 generic time, in S.I:", converter.to_si(1 | nbody_system.time).in_(units.yr))
print("10 km/s, in generic:", converter.to_nbody(10.0 | units.km / units.s))

In [ ]:
converter = generic_unit_converter.ConvertBetweenGenericAndSiUnits(1 | units.MSun, 1 | units.AU, 1 | units.yr)
print("Mass of the sun, scaled:", converter.to_nbody(1 | units.MSun))
print("10 generic masses, in S.I.:", converter.to_si(10 | nbody_system.mass))
print("1 generic time, in S.I:", converter.to_si(1 | nbody_system.time).in_(units.yr))
print("10 km/s, in generic:", converter.to_nbody(10.0 | units.km / units.s))

Specifying a length twice or specifying a speed and a length and a time will result in an error. 

In [ ]:
generic_unit_converter.ConvertBetweenGenericAndSiUnits(
    1 | units.MSun, 
    1 | units.AU, 
    1 | units.m,
)

As an example, the following defines a converter for Planck units:

In [ ]:
natural_units_convert = generic_unit_converter.ConvertBetweenGenericAndSiUnits(
    constants.c,
    constants.G,
    constants.hbar,
    1/(4*numpy.pi*constants.eps0),
    constants.kB,
)

M = 1 | generic_unit_system.mass
T = 1 | generic_unit_system.time
L = 1 | generic_unit_system.length
Q = 1 | generic_unit_system.charge
THETA = 1 | generic_unit_system.temperature

print(natural_units_convert.to_si(M).in_(units.kg))
print(natural_units_convert.to_si(T).in_(units.s))
print(natural_units_convert.to_si(L).in_(units.m))
print(natural_units_convert.to_si(Q).in_(units.C))
print(natural_units_convert.to_si(THETA).in_(units.K))

Of course unit commensurability is still enforced:

In [ ]:
print((10.0 | nbody_system.length) + (10.0 | generic_unit_system.time))